## Modeling Exercises

#### Select a dataset with a continuous target variable.
> - I'm opting to utilize the `zillow` dataset because it will be part of the project.
> - This data has been properly prepared in previous exercises.
> - Work through all of the steps outlined in the lesson, from setting the baseline to selected a model and evaluating the final model on your test data.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pandas as pd

from sklearn.model_selection import train_test_split

from scipy.stats import pearsonr, spearmanr

from acquire import *
from wrangle import *
from explore import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = get_zillow_data()


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152864 entries, 0 to 2152863
Data columns (total 12 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   latitude                      float64
 4   longitude                     float64
 5   taxvaluedollarcnt             float64
 6   yearbuilt                     float64
 7   taxamount                     float64
 8   fips                          float64
 9   regionidzip                   float64
 10  regionidcity                  float64
 11  lotsizesquarefeet             float64
dtypes: float64(12)
memory usage: 213.5 MB


In [29]:
from pyzipcode import ZipCodeDatabase
from geopy.geocoders import Nominatim
from geopy.point import Point
import csv
zcdb = ZipCodeDatabase()
geolocator = Nominatim(user_agent='geoaipExercises')

(2152864, 12)

In [70]:
location_df = pd.read_csv('location_and_zillow_data.csv')
unlocated_df = df[~((df.latitude.isin(location_df.latitude)) & (df.longitude.isin(location_df.longitude)))]
# population mapping 
popdf = pd.read_csv('2010_Census_Population_By_Zipcode.csv')

In [77]:
while unlocated_df.shape[0] != 0:
    location_df = pd.read_csv('location_and_zillow_data.csv')
    print(location_df.shape)
    # Checks to see if the data has been found
    unlocated_df = df[~((df.latitude.isin(location_df.latitude)) & (df.longitude.isin(location_df.longitude)))]
    print(unlocated_df.shape[0])
    location_df.head()
    cycle(unlocated_df, location_df)
    print('another cycle')

(650, 37)
2151461
another cycle
(750, 37)
2151360
another cycle
(850, 37)
2151260
another cycle
(950, 39)
2151088
another cycle
(1050, 39)
2150988


KeyboardInterrupt: 

In [76]:
def cycle(unlocated_df, location_df):
    temp_df = unlocated_df.head(100)

    def location_lookup(latitude, longitude):
        lat = str(int(latitude))[:-6]+'.'+str(int(latitude))[-6:] 
        long = str(int(longitude))[:-6]+'.'+str(int(longitude))[-6:] 
        loc = lat + ',' + long
        location = geolocator.reverse(loc)
        location_dict = location.raw['address']
        return location_dict

    temp_df['location'] = temp_df.apply(lambda x: location_lookup(x.latitude, x.longitude), axis=1)
    locations = temp_df.location.apply(pd.Series)
    new_df = pd.merge(temp_df, locations, left_index=True, right_index=True)

    # Set index to zipcode
    tdf = popdf.set_index('zipcode')
    # make the zipcode a dict value to map
    pop = popdf.to_dict()
    # set population dict to be the population so it's formatted properly
    population = pop['population']
    # set population column to be the postcode to be mapped too
    new_df['population'] = new_df.postcode
    # replace the zipcode with the population information
    new_df.population.update(pd.Series(population))
    new_df.head()
    location_df = location_df.append(new_df)
    location_df.to_csv('location_and_zillow_data.csv', index=False)